In [1]:
import os
from pathlib import Path
import numpy as np
import pandas as pd
import bifacial_radiance as br

# testfolder = Path().resolve().parent.parent / 'TEMP' / 'bifacial_radiance'

testfolder = os.path.join('TEMP')

# if not os.path.exists(testfolder):
#     os.makedirs(testfolder)
    
# print ("Your simulation will be stored in %s" % testfolder)

In [2]:
print(os.path.join('TEMP', 'BARN_validation_jan18_2023'))

TEMP\BARN_validation_jan18_2023


In [3]:
# remove unwanted results, in this case all the "fronts"
filesall_west = os.listdir(os.path.join(testfolder,'BARN_valid_inchwest_feb2_2023','results','oneinchwest')) # How do I save the path to an object so that I don't have to repeat the whole process?
filesall_east = os.listdir(os.path.join(testfolder,'BARN_valid_inchwest_feb2_2023','results','oneincheast'))
filesall_origin = os.listdir(os.path.join(testfolder,'BARN_validation_jan18_2023','results'))
filestoclean_west = [e for e in filesall_west if e.endswith('_Front.csv')]
for cc in range(0, len(filestoclean_west)):
    filetoclean_west = filestoclean_west[cc]
    os.remove(os.path.join(testfolder,'BARN_valid_inchwest_feb2_2023','results','oneinchwest', filetoclean_west))

filestoclean_east = [e for e in filesall_east if e.endswith('_Front.csv')]
for cc in range(0, len(filestoclean_east)):
    filetoclean_east = filestoclean_east[cc]
    os.remove(os.path.join(testfolder,'BARN_valid_inchwest_feb2_2023','results','oneincheast', filetoclean_east))

filestoclean_origin = [e for e in filesall_origin if e.endswith('_Front.csv')]
for cc in range(0, len(filestoclean_origin)):
    filetoclean_origin = filestoclean_origin[cc]
    os.remove(os.path.join(testfolder,'BARN_validation_jan18_2023','results', filetoclean_origin))

In [4]:
print(filesall_west)
print(filesall_east)
print(filesall_origin)

['irr_1axis_2023-09-20_0600Ground_Row4_Module7_Back.csv', 'irr_1axis_2023-09-20_0615Ground_Row4_Module7_Back.csv', 'irr_1axis_2023-09-20_0630Ground_Row4_Module7_Back.csv', 'irr_1axis_2023-09-20_0645Ground_Row4_Module7_Back.csv', 'irr_1axis_2023-09-20_0700Ground_Row4_Module7_Back.csv', 'irr_1axis_2023-09-20_0715Ground_Row4_Module7_Back.csv', 'irr_1axis_2023-09-20_0730Ground_Row4_Module7_Back.csv', 'irr_1axis_2023-09-20_0745Ground_Row4_Module7_Back.csv', 'irr_1axis_2023-09-20_0800Ground_Row4_Module7_Back.csv', 'irr_1axis_2023-09-20_0815Ground_Row4_Module7_Back.csv', 'irr_1axis_2023-09-20_0830Ground_Row4_Module7_Back.csv', 'irr_1axis_2023-09-20_0845Ground_Row4_Module7_Back.csv', 'irr_1axis_2023-09-20_0900Ground_Row4_Module7_Back.csv', 'irr_1axis_2023-09-20_0915Ground_Row4_Module7_Back.csv', 'irr_1axis_2023-09-20_0930Ground_Row4_Module7_Back.csv', 'irr_1axis_2023-09-20_0945Ground_Row4_Module7_Back.csv', 'irr_1axis_2023-09-20_1000Ground_Row4_Module7_Back.csv', 'irr_1axis_2023-09-20_1015Grou

In [5]:
from datetime import datetime
import re
print((os.path.join(testfolder, 'results')))


TEMP\results


## Read in the results
Below script reads in the results of the bifacial radiance modeling for a specific date and appends them to arrays that will then be joined into a dataframe.

In [6]:
filestarter = "irr_1axis_2023-09" # The string your results file names start with.
filelist_west = os.listdir(os.path.join(testfolder,'BARN_valid_inchwest_feb2_2023','results','oneinchwest'))
filelist_east = os.listdir(os.path.join(testfolder,'BARN_valid_inchwest_feb2_2023','results','oneincheast'))
filelist_origin = os.listdir(os.path.join(testfolder,'BARN_validation_jan18_2023','results'))
prefixed = [filename for filename in filelist_west if filename.startswith(filestarter)]
distance = []
groundirrad = []
filenamed = []
faillist = []
Datetime = []
Ap_1 = []
Ap_2 = []
Ap_3 = []
Ap_4 = []
Ap_5 = []

print('{} files in the directory'.format(filelist_west.__len__()))
print('{} groundscan files in the directory'.format(prefixed.__len__()))
i = 0  # counter to track # files loaded.

for i in range (0, len(prefixed)):
    ind = prefixed[i].split('_')

    try:
        Datetime.append(re.search("([0-9]{4}\-[0-9]{2}\-[0-9]{2}\_[0-9]{4})", prefixed[i])[0])
        resultsDF = br.load.read1Result(os.path.join(testfolder, 'BARN_valid_inchwest_feb2_2023','results','oneinchwest', prefixed[i]))
        Ap_1.append(resultsDF['Wm2Back'][0])
        Ap_2.append(resultsDF['Wm2Back'][1])
        Ap_3.append(resultsDF['Wm2Back'][2])
        Ap_4.append(resultsDF['Wm2Back'][3])
        Ap_5.append(resultsDF['Wm2Back'][4])

        filenamed.append(prefixed[i])
    except:
        print(" FAILED ", i, prefixed[i])
        faillist.append(prefixed[i])
        

west = pd.DataFrame(list(zip(Ap_1, Ap_2, Ap_3, Ap_4, Ap_5)), columns = ['Ap_1', 'Ap_2', 'Ap_3', 'Ap_4', 'Ap_5'])

west['Datetime'] = Datetime
west['Datetime'] = pd.to_datetime(west['Datetime'], format ='%Y-%m-%d_%H%M')

filelist_east = os.listdir(os.path.join(testfolder,'BARN_valid_inchwest_feb2_2023','results','oneincheast'))

prefixed = [filename for filename in filelist_east if filename.startswith(filestarter)]
distance = []
groundirrad = []
filenamed = []
faillist = []
Datetime = []
Ap_1 = []
Ap_2 = []
Ap_3 = []
Ap_4 = []
Ap_5 = []

print('{} files in the directory'.format(filelist_east.__len__()))
print('{} groundscan files in the directory'.format(prefixed.__len__()))
i = 0  # counter to track # files loaded.

for i in range (0, len(prefixed)):
    ind = prefixed[i].split('_')

    try:
        Datetime.append(re.search("([0-9]{4}\-[0-9]{2}\-[0-9]{2}\_[0-9]{4})", prefixed[i])[0])
        resultsDF = br.load.read1Result(os.path.join(testfolder, 'BARN_valid_inchwest_feb2_2023','results','oneincheast', prefixed[i]))
        Ap_1.append(resultsDF['Wm2Back'][0])
        Ap_2.append(resultsDF['Wm2Back'][1])
        Ap_3.append(resultsDF['Wm2Back'][2])
        Ap_4.append(resultsDF['Wm2Back'][3])
        Ap_5.append(resultsDF['Wm2Back'][4])

        filenamed.append(prefixed[i])
    except:
        print(" FAILED ", i, prefixed[i])
        faillist.append(prefixed[i])
        

east = pd.DataFrame(list(zip(Ap_1, Ap_2, Ap_3, Ap_4, Ap_5)), columns = ['Ap_1', 'Ap_2', 'Ap_3', 'Ap_4', 'Ap_5'])

east['Datetime'] = Datetime
east['Datetime'] = pd.to_datetime(east['Datetime'], format ='%Y-%m-%d_%H%M')


filelist_origin = os.listdir(os.path.join(testfolder,'BARN_validation_jan18_2023','results'))

prefixed = [filename for filename in filelist_origin if filename.startswith(filestarter)]
distance = []
groundirrad = []
filenamed = []
faillist = []
Datetime = []
Ap_1 = []
Ap_2 = []
Ap_3 = []
Ap_4 = []
Ap_5 = []

print('{} files in the directory'.format(filelist_origin.__len__()))
print('{} groundscan files in the directory'.format(prefixed.__len__()))
i = 0  # counter to track # files loaded.

for i in range (0, len(prefixed)):
    ind = prefixed[i].split('_')

    try:
        Datetime.append(re.search("([0-9]{4}\-[0-9]{2}\-[0-9]{2}\_[0-9]{4})", prefixed[i])[0])
        resultsDF = br.load.read1Result(os.path.join(testfolder, 'BARN_validation_jan18_2023','results', prefixed[i]))
        Ap_1.append(resultsDF['Wm2Back'][0])
        Ap_2.append(resultsDF['Wm2Back'][1])
        Ap_3.append(resultsDF['Wm2Back'][2])
        Ap_4.append(resultsDF['Wm2Back'][3])
        Ap_5.append(resultsDF['Wm2Back'][4])

        filenamed.append(prefixed[i])
    except:
        print(" FAILED ", i, prefixed[i])
        faillist.append(prefixed[i])
        

origin = pd.DataFrame(list(zip(Ap_1, Ap_2, Ap_3, Ap_4, Ap_5)), columns = ['Ap_1', 'Ap_2', 'Ap_3', 'Ap_4', 'Ap_5'])

origin['Datetime'] = Datetime
origin['Datetime'] = pd.to_datetime(origin['Datetime'], format ='%Y-%m-%d_%H%M')


451 files in the directory
451 groundscan files in the directory
451 files in the directory
451 groundscan files in the directory
451 files in the directory
451 groundscan files in the directory


In [14]:
modeled_l = pd.concat([west.assign(offset = "west"), east.assign(offset = "east"), origin.assign(offset = "origin")])
modeled_ll = pd.melt(modeled_l, id_vars = ["Datetime", "offset"], value_vars = ["Ap_1", "Ap_2", "Ap_3", "Ap_4", "Ap_5"], value_name = "modeled")#.set_index(["Datetime", "offset"])
print(modeled_ll)

west = west.set_index('Datetime').add_suffix(".west")
east = east.set_index('Datetime').add_suffix(".east")
origin = origin.set_index('Datetime').add_suffix(".origin")

modeled = pd.concat([west, east, origin], axis = 1, join = 'inner')


# print(modeled_l)

                Datetime  offset variable    modeled
0    2023-09-20 06:00:00    west     Ap_1   9.043741
1    2023-09-20 06:15:00    west     Ap_1  10.280010
2    2023-09-20 06:30:00    west     Ap_1  14.261810
3    2023-09-20 06:45:00    west     Ap_1  16.918750
4    2023-09-20 07:00:00    west     Ap_1  19.291870
...                  ...     ...      ...        ...
6760 2023-09-29 09:45:00  origin     Ap_5  25.096170
6761 2023-09-29 10:00:00  origin     Ap_5  25.526720
6762 2023-09-29 10:15:00  origin     Ap_5  24.730680
6763 2023-09-29 10:30:00  origin     Ap_5  25.092370
6764 2023-09-29 10:45:00  origin     Ap_5  26.698430

[6765 rows x 4 columns]


In [11]:
measured = pd.read_csv(os.path.join(os.path.join(Path().resolve(), 'Data','BARNirrad_measured.csv')), header = 1) # The field pyranometer data
measured['Datetime'] = pd.to_datetime(measured['TIMESTAMP'], format ='%m/%d/%Y %H:%M')
measured = measured.drop(["TIMESTAMP", "RECORD"], axis = 'columns') # drop this column so the resample command does not confuse TIMESTAMP and Datetime columns
measured = measured.set_index('Datetime').resample('15T', axis = 'index', label='right', closed='right').mean() # The "label" and the "closed" arguments are different from that of how the SRRL data was read-in (which was "right" for both arguments), but this somehow yields a better timing match.
measured_l = measured.reset_index()[["Datetime","Ap_1", "Ap_2", "Ap_3", "Ap_4", "Ap_5"]].melt(id_vars= "Datetime", value_vars = ["Ap_1", "Ap_2", "Ap_3", "Ap_4", "Ap_5"], value_name = "measured")

In [13]:
print(modeled_ll)
print(measured_l)
modeledandmeasured_pyr = pd.concat([modeled_ll.reset_index().set_index("Datetime"), measured_l.set_index("Datetime")], axis=1, join = "inner")
modeledandmeasured_pyr = pd.concat([modeled_ll.set_index("Datetime"), measured_l.set_index("Datetime")], axis=0, join = "inner")

                Datetime  offset variable    modeled
0    2023-09-20 06:00:00    west     Ap_1   9.043741
1    2023-09-20 06:15:00    west     Ap_1  10.280010
2    2023-09-20 06:30:00    west     Ap_1  14.261810
3    2023-09-20 06:45:00    west     Ap_1  16.918750
4    2023-09-20 07:00:00    west     Ap_1  19.291870
...                  ...     ...      ...        ...
6760 2023-09-29 09:45:00  origin     Ap_5  25.096170
6761 2023-09-29 10:00:00  origin     Ap_5  25.526720
6762 2023-09-29 10:15:00  origin     Ap_5  24.730680
6763 2023-09-29 10:30:00  origin     Ap_5  25.092370
6764 2023-09-29 10:45:00  origin     Ap_5  26.698430

[6765 rows x 4 columns]
                 Datetime variable   measured
0     2023-09-07 11:30:00     Ap_1  41.392173
1     2023-09-07 11:45:00     Ap_1  41.385736
2     2023-09-07 12:00:00     Ap_1  40.700726
3     2023-09-07 12:15:00     Ap_1  39.961025
4     2023-09-07 12:30:00     Ap_1  39.547040
...                   ...      ...        ...
26650 2023-11-01 

In [50]:
# Index the modeled data to the datetime column and add suffix before concatenating with the measured data. This will allow the "inner" join of the two dataframes using the indexed datetime and easy pivoting of the data afterwards.
# modeled = resultsdf.set_index('Datetime')
modeled_suffix = modeled.add_suffix(".modeled")
modeled['datatype']="modeled"

# Load the measured data
measured = pd.read_csv(os.path.join(os.path.join(Path().resolve(), 'Data','BARNirrad_measured.csv')), header = 1) # The field pyranometer data
srrl_weather = pd.read_csv(str(Path().resolve() / 'WeatherFiles' /  'PSM3_15T.csv'), header = 2) # Data from SRRL

# Create a Datetime variable and set it as the index
srrl_weather['date'] = srrl_weather[['Year','Month','Day']].astype(str).agg('-'.join, axis=1) # Couldn't think of a proper way to chain this process into a single command. If you think of something, please let me know
srrl_weather['time'] = srrl_weather[['Hour','Minute']].astype(str).agg(':'.join, axis=1)
srrl_weather['Datetime'] = pd.to_datetime(srrl_weather[['date','time']].agg(" ".join, axis=1))
srrl_weather.set_index('Datetime', inplace=True)

# Create a Datetime variable for the measured data using the existing 'TIMESTAMP' column.pivot

measured['Datetime'] = pd.to_datetime(measured['TIMESTAMP'], format ='%m/%d/%Y %H:%M')
measured = measured.drop(["TIMESTAMP", "RECORD"], axis = 'columns') # drop this column so the resample command does not confuse TIMESTAMP and Datetime columns
measured = measured.set_index('Datetime').resample('15T', axis = 'index', label='right', closed='right').mean() # The "label" and the "closed" arguments are different from that of how the SRRL data was read-in (which was "right" for both arguments), but this somehow yields a better timing match.

# Add suffix to the columns in the measured dataframe before concatenating it with the modeled dataframe
measured_suffix = measured.add_suffix(".measured")
measured["datatype"]="measured"

modeledandmeasured_suffix = pd.concat([modeled_suffix, measured_suffix], axis=1, join='inner')
modeledandmeasured_melt = pd.wide_to_long(df = modeledandmeasured_suffix.reset_index(), stubnames= ["Ap_1", "Ap_2", "Ap_3", "Ap_4", "Ap_5","ST_con", "ST_pv_bp", "ST_pv_is", "PAR_con", "PAR_pv", "Batt_Volt", "PTemp_C"], i = "Datetime", j = "datatype", sep = ".", suffix = "\D+").reset_index().set_index('Datetime')

modeledandmeasured_melt2 = modeledandmeasured_melt[['datatype', 'Ap_1', 'Ap_2', 'Ap_3', 'Ap_4', 'Ap_5']].reset_index().melt(id_vars = ['Datetime', 'datatype'], var_name = 'position', value_name= 'value').reset_index()
modeledandmeasured_melt3 = modeledandmeasured_melt2.pivot(index=['Datetime', 'position'], columns = 'datatype', values = 'value')


# Sum up the irradiance values and divide it by 4 to get accumulated daily radiation
modeledandmeasured_sum = modeledandmeasured_melt2.reset_index().groupby(['position', 'datatype']).agg({"value":"sum"})['value'].transform(lambda x:x/4)

# Grab just the measured
ghipar = modeledandmeasured_melt.loc[modeledandmeasured_melt['datatype']=='measured'][['Ap_3', 'PAR_pv', 'PAR_con']]
srrlandghipar = pd.concat([ghipar, srrl_weather], axis=1, join="inner")[['Ap_3','GHI', 'PAR_pv', 'PAR_con']].rename(columns={'Ap_3':'ghi_pv','GHI':'ghi_con','PAR_pv':'par_pv','PAR_con':'par_con'}).reset_index()
srrlandghipar = pd.wide_to_long(df = srrlandghipar.reset_index(), stubnames=['ghi','par'], i = "Datetime", j = "location", sep = "_", suffix = "\D+").reset_index()
print(srrlandghipar)


               Datetime location  index         ghi          par
0   2023-09-20 06:00:00       pv      0    9.809481    22.695676
1   2023-09-20 06:15:00       pv      1   31.131996    73.920773
2   2023-09-20 06:30:00       pv      2   28.761644    82.067543
3   2023-09-20 06:45:00       pv      3   23.257370    76.957432
4   2023-09-20 07:00:00       pv      4   41.063726   109.636165
..                  ...      ...    ...         ...          ...
897 2023-09-29 09:45:00      con    446  647.789333  1576.165000
898 2023-09-29 10:00:00      con    447  677.646867  1669.476733
899 2023-09-29 10:15:00      con    448  709.302267  1753.681200
900 2023-09-29 10:30:00      con    449  734.460333  1819.112867
901 2023-09-29 10:45:00      con    450  754.240867  1871.513600

[902 rows x 5 columns]


In [10]:
# look at residuals (MBD, RMSE) based on Grear, Gpoa and Gtotal_modeled. From B. Marion Solar Energy 2016
def MBD(meas,model):
    # MBD=100∙[((1⁄(m)∙∑〖(y_i-x_i)]÷[(1⁄(m)∙∑〖x_i]〗)〗)
    import pandas as pd
    df = pd.DataFrame({'model':model,'meas':meas})
    # rudimentary filtering of modeled irradiance
    df = df.dropna()
    minirr = meas.min()
    df = df[df.model>minirr]
    m = df.__len__()
    out = 100*((1/m)*sum(df.model-df.meas))/df.meas.mean()
    return out

def RMSE(meas,model):
    #RMSD=100∙〖[(1⁄(m)∙∑▒(y_i-x_i )^2 )]〗^(1⁄2)÷[(1⁄(m)∙∑▒〖x_i]〗)
    import numpy as np
    import pandas as pd
    df = pd.DataFrame({'model':model,'meas':meas})
    df = df.dropna()
    minirr = meas.min()
    df = df[df.model>minirr]
    m = df.__len__()
    out = 100*np.sqrt(1/m*sum((df.model-df.meas)**2))/df.meas.mean()
    return out

# residuals absolute output (not %) 
def MBD_abs(meas,model):
    # MBD=100∙[((1⁄(m)∙∑〖(y_i-x_i)]÷[(1⁄(m)∙∑〖x_i]〗)〗)
    import pandas as pd
    df = pd.DataFrame({'model':model,'meas':meas})
    # rudimentary filtering of modeled irradiance
    df = df.dropna()
    minirr = meas.min()
    df = df[df.model>minirr]
    m = df.__len__()
    out = ((1/m)*sum(df.model-df.meas))
    return out

def RMSE_abs(meas,model):
    #RMSD=100∙〖[(1⁄(m)∙∑▒(y_i-x_i )^2 )]〗^(1⁄2)÷[(1⁄(m)∙∑▒〖x_i]〗)
    import numpy as np
    import pandas as pd
    df = pd.DataFrame({'model':model,'meas':meas})
    df = df.dropna()
    minirr = meas.min()
    df = df[df.model>minirr]
    m = df.__len__()
    out = np.sqrt(1/m*sum((df.model-df.meas)**2))
    return out

In [11]:
# Create the empty arrays (or lists? I don't know all the object types in python) to save the result of the looped error analysis
MBD_result_west = []
RMSE_result_west = []
MBD_abs_result_west = []
RMSE_abs_result_west = []

MBD_result_east = []
RMSE_result_east = []
MBD_abs_result_east = []
RMSE_abs_result_east = []

MBD_result_origin = []
RMSE_result_origin = []
MBD_abs_result_origin = []
RMSE_abs_result_origin = []
faillist = []

# Grab the list of column names
colnames = list(modeledandmeasured_suffix.columns.values)
print(colnames)
modposition_west = []
modposition_east = []
modposition_origin = []
measposition = []

# Setting the index to 0 prior to running the for-loop
i = 0

# For-looping the error analyses
for i in range (0, 5): # The index range was determined after looking at the 
    try:
        MBD_result_west.append(MBD(modeledandmeasured_suffix[colnames[i+15]], modeledandmeasured_suffix[colnames[i]])) # The modeled columns and the measured counterparts are five columns apart, hence "i" and "i+5"
        RMSE_result_west.append(RMSE(modeledandmeasured_suffix[colnames[i+15]], modeledandmeasured_suffix[colnames[i]]))
        MBD_abs_result_west.append(MBD_abs(modeledandmeasured_suffix[colnames[i+15]], modeledandmeasured_suffix[colnames[i]]))
        RMSE_abs_result_west.append(RMSE_abs(modeledandmeasured_suffix[colnames[i+15]], modeledandmeasured_suffix[colnames[i]]))

        MBD_result_east.append(MBD(modeledandmeasured_suffix[colnames[i+15]], modeledandmeasured_suffix[colnames[i+5]])) # The modeled columns and the measured counterparts are five columns apart, hence "i" and "i+5"
        RMSE_result_east.append(RMSE(modeledandmeasured_suffix[colnames[i+15]], modeledandmeasured_suffix[colnames[i+5]]))
        MBD_abs_result_east.append(MBD_abs(modeledandmeasured_suffix[colnames[i+15]], modeledandmeasured_suffix[colnames[i+5]]))
        RMSE_abs_result_east.append(RMSE_abs(modeledandmeasured_suffix[colnames[i+15]], modeledandmeasured_suffix[colnames[i+5]]))

        MBD_result_origin.append(MBD(modeledandmeasured_suffix[colnames[i+15]], modeledandmeasured_suffix[colnames[i+10]])) # The modeled columns and the measured counterparts are five columns apart, hence "i" and "i+5"
        RMSE_result_origin.append(RMSE(modeledandmeasured_suffix[colnames[i+15]], modeledandmeasured_suffix[colnames[i+10]]))
        MBD_abs_result_origin.append(MBD_abs(modeledandmeasured_suffix[colnames[i+15]], modeledandmeasured_suffix[colnames[i+10]]))
        RMSE_abs_result_origin.append(RMSE_abs(modeledandmeasured_suffix[colnames[i+15]], modeledandmeasured_suffix[colnames[i+10]]))

        modposition_west.append(colnames[i])
        modposition_east.append(colnames[i+5])
        modposition_origin.append(colnames[i + 10])
        measposition.append(colnames[i+15])
    except:
        print(" FAILED ", i, colnames[i])
        faillist.append(colnames[i])

# Join the arrays into a dataframe
validationdf = pd.DataFrame(list(zip(modposition_west, modposition_east, modposition_origin, measposition, 
                                     MBD_result_west, RMSE_result_west, MBD_abs_result_west, RMSE_abs_result_west,
                                     MBD_result_east, RMSE_result_east, MBD_abs_result_east, RMSE_abs_result_east,
                                     MBD_result_origin, RMSE_result_origin, MBD_abs_result_origin, RMSE_abs_result_origin)), 
                                     columns = ['Modeled_west', 'Modeled_east','Modeled_origin','Measured', 
                                                'MBD_west', 'RMSE_west', 'MBD_abs_west', 'RMSE_abs_west',
                                                'MBD_east', 'RMSE_east', 'MBD_abs_east', 'RMSE_abs_east',
                                                'MBD_origin', 'RMSE_origin', 'MBD_abs_origin', 'RMSE_abs_origin'])    

# Export the result into a csv
validationdf.to_csv('RMSE_MDE.csv',index=False)
print(validationdf)

['Ap_1_west.modeled', 'Ap_2_west.modeled', 'Ap_3_west.modeled', 'Ap_4_west.modeled', 'Ap_5_west.modeled', 'Ap_1_east.modeled', 'Ap_2_east.modeled', 'Ap_3_east.modeled', 'Ap_4_east.modeled', 'Ap_5_east.modeled', 'Ap_1_origin.modeled', 'Ap_2_origin.modeled', 'Ap_3_origin.modeled', 'Ap_4_origin.modeled', 'Ap_5_origin.modeled', 'Ap_1.measured', 'Ap_2.measured', 'Ap_3.measured', 'Ap_4.measured', 'Ap_5.measured', 'ST_con.measured', 'ST_pv_bp.measured', 'ST_pv_is.measured', 'PAR_con.measured', 'PAR_pv.measured', 'Batt_Volt.measured', 'PTemp_C.measured']
        Modeled_west       Modeled_east       Modeled_origin       Measured  \
0  Ap_1_west.modeled  Ap_1_east.modeled  Ap_1_origin.modeled  Ap_1.measured   
1  Ap_2_west.modeled  Ap_2_east.modeled  Ap_2_origin.modeled  Ap_2.measured   
2  Ap_3_west.modeled  Ap_3_east.modeled  Ap_3_origin.modeled  Ap_3.measured   
3  Ap_4_west.modeled  Ap_4_east.modeled  Ap_4_origin.modeled  Ap_4.measured   
4  Ap_5_west.modeled  Ap_5_east.modeled  Ap_5_origin

In [12]:
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
# Set the size for each figure, in pixels
figwidth = 1000
figheight = 400

# Simple line graph of the measured vs modeled irradiance values
fig1 = px.line()

## The following figures are best displayed in wide-screen.

In [ ]:
# Set the size for each figure, in pixels
figwidth = 1000
figheight = 400

# Simple line graph of the measured vs modeled irradiance values
fig1 = px.line(modeledandmeasured_melt2, x = 'Datetime', y = 'value', color = 'datatype', facet_row = 'position', facet_col_wrap = 1, width = figwidth, height = 1000)
fig1.update_layout(
    plot_bgcolor='white',
    xaxis_title = "",
    yaxis_title = "15-min average irradiance (W/m<sup>2</sup>)",
    legend=dict(
        yanchor="top",
        y=1.2,
        xanchor="center",
        x=0.5
    ),
    legend_title=dict(
        text = "Data type"
    ),
    legend_orientation="h"
)
fig1.update_xaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey',
    title = " "
)
fig1.update_yaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey'
)
# for axis in fig1.layout:
#    if axis.startswith("xaxis"):
#       fig1.layout[axis].title = ""
fig1.show()

rsqval = []
# 1-to-1 plot of the measured vs modeled irradiance values
fig2 = px.scatter(modeledandmeasured_melt3.reset_index(), x = 'measured', y = 'modeled', facet_col = 'position', facet_col_wrap=5, trendline="ols", trendline_color_override="black", width = figwidth, height = figheight)
    # Accessing the R-squared values and annotating each facet
for i in range(len(np.unique(modeledandmeasured_melt3.reset_index()['position']))): #'position' can be replaced with whatever column is being used for the facets
    # Accessing R-squared value
    rsqval=round(px.get_trendline_results(fig2).iloc[i]["px_fit_results"].rsquared, 4)

    # Annotating each facet with the R-squared value
    fig2.add_annotation(
        xref='paper', yref='paper',
        x=10, y=800,
        xanchor='left', yanchor='bottom',
        text=f'R<sup>2</sup>: {rsqval:.4f}',
        showarrow=False,
        font=dict(size=12),
        row= 1, col=i+1 # only works when the facets are in a single row
    )

fig2.update_layout(
    plot_bgcolor='white',
    xaxis_title = "Measured irradiance (W/m<sup>2</sup>)",
    yaxis_title = "Modeled irradiance (W/m<sup>2</sup>)"
)
fig2.update_xaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey'
)
fig2.update_yaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey'
)
fig2.update_traces(marker=dict(size=6,
                                color='rgba(135, 206, 250, 0.5)',
                                line=dict(width=2,
                                    color='DarkSlateGrey')),
                  selector=dict(mode='markers'))
fig2.show()

# side-by-side barcharts
fig3 = px.bar(modeledandmeasured_sum.reset_index(), x='position', y='value', color='datatype', barmode='group', width = figwidth, height = figheight)
fig3.update_layout(
    plot_bgcolor='white',
    xaxis_title = "",
    yaxis_title = "Daily total radiation (Wh/m<sup>2</sup>)",
    legend=dict(
        yanchor="top",
        y=0.95,
        xanchor="left",
        x=0.01
    ),
    legend_title=dict(
        text = "Data type"
    ),
    legend_orientation="h"
)
fig3.update_xaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey'
)
fig3.update_yaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey'
)
fig3.show()

# GHI vs PAR inside an array vs 
fig4 = px.scatter(srrlandghipar, x = 'ghi', y = 'par', color = 'location', trendline="ols",color_discrete_sequence=px.colors.qualitative.Antique, width = figwidth, height = figheight)

fig4.update_layout(
    legend=dict(
        yanchor="top",
        y=0.95,
        xanchor="left",
        x=0.01
        ),
    legend_title=dict(
        text = "Location"
        ),
    legend_orientation="h",
    plot_bgcolor='white',
    yaxis_title = "Measured photoactive radiation (umol m<sup>-2</sup> s<sup>-1</sup>)",
    xaxis_title = "Measured global horizontal irradiance (W/m<sup>2</sup>)"
)
fig4.update_xaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey'
)
fig4.update_yaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey'
)
fig4.update_traces(marker=dict(size=6,
                                # color='rgba(135, 206, 250, 0.5)',
                                line=dict(width=2,
                                    color='DarkSlateGrey')),
                  selector=dict(mode='markers'))


## Stitch all the figures into a single HTML
Function below taken from https://stackoverflow.com/questions/45577255/plot-multiple-figures-as-subplots

In [ ]:
def figures_to_html(figs, filename="dashboard.html"):
    with open(filename, 'w') as dashboard:
        dashboard.write("<html><head></head><body>" + "\n")
        for fig in figs:
            inner_html = fig.to_html().split('<body>')[1].split('</body>')[0]
            dashboard.write(inner_html)
        dashboard.write("</body></html>" + "\n")


figures_to_html([fig1, fig2, fig3, fig4], 'BARN_validation.html')

In [ ]:
print(modeledandmeasured_melt2)

In [ ]:
# Simple line graph of the measured vs modeled irradiance values
fig1 = px.line(modeledandmeasured_melt2.loc[modeledandmeasured_melt2['position'] == 'Ap_1'], x = 'Datetime', y = 'value', color = 'datatype', facet_row = 'position', facet_col_wrap = 1, width = figwidth, height = figheight)
fig1.update_layout(
    plot_bgcolor='white',
    xaxis_title = "",
    yaxis_title = "15-min average irradiance (W/m<sup>2</sup>)",
    legend=dict(
        yanchor="top",
        y=1.2,
        xanchor="center",
        x=0.5
    ),
    legend_title=dict(
        text = "Data type"
    ),
    legend_orientation="h"
)
fig1.update_xaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey',
    title = " "
)
fig1.update_yaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey'
)
# for axis in fig1.layout:
#    if axis.startswith("xaxis"):
#       fig1.layout[axis].title = ""
fig1.show()

fig2 = px.line(modeledandmeasured_melt2.loc[modeledandmeasured_melt2['position'] == 'Ap_2'], x = 'Datetime', y = 'value', color = 'datatype', facet_row = 'position', facet_col_wrap = 1, width = figwidth, height = figheight)
fig2.update_layout(
    plot_bgcolor='white',
    xaxis_title = "",
    yaxis_title = "15-min average irradiance (W/m<sup>2</sup>)",
    legend=dict(
        yanchor="top",
        y=1.2,
        xanchor="center",
        x=0.5
    ),
    legend_title=dict(
        text = "Data type"
    ),
    legend_orientation="h"
)
fig2.update_xaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey',
    title = " "
)
fig2.update_yaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey'
)
# for axis in fig1.layout:
#    if axis.startswith("xaxis"):
#       fig1.layout[axis].title = ""
fig2.show()

fig3 = px.line(modeledandmeasured_melt2.loc[modeledandmeasured_melt2['position'] == 'Ap_3'], x = 'Datetime', y = 'value', color = 'datatype', facet_row = 'position', facet_col_wrap = 1, width = figwidth, height = figheight)
fig3.update_layout(
    plot_bgcolor='white',
    xaxis_title = "",
    yaxis_title = "15-min average irradiance (W/m<sup>2</sup>)",
    legend=dict(
        yanchor="top",
        y=1.2,
        xanchor="center",
        x=0.5
    ),
    legend_title=dict(
        text = "Data type"
    ),
    legend_orientation="h"
)
fig3.update_xaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey',
    title = " "
)
fig3.update_yaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey'
)
# for axis in fig1.layout:
#    if axis.startswith("xaxis"):
#       fig1.layout[axis].title = ""
fig3.show()

fig4 = px.line(modeledandmeasured_melt2.loc[modeledandmeasured_melt2['position'] == 'Ap_4'], x = 'Datetime', y = 'value', color = 'datatype', facet_row = 'position', facet_col_wrap = 1, width = figwidth, height = figheight)
fig4.update_layout(
    plot_bgcolor='white',
    xaxis_title = "",
    yaxis_title = "15-min average irradiance (W/m<sup>2</sup>)",
    legend=dict(
        yanchor="top",
        y=1.2,
        xanchor="center",
        x=0.5
    ),
    legend_title=dict(
        text = "Data type"
    ),
    legend_orientation="h"
)
fig4.update_xaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey',
    title = " "
)
fig4.update_yaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey'
)
# for axis in fig1.layout:
#    if axis.startswith("xaxis"):
#       fig1.layout[axis].title = ""
fig4.show()

fig5 = px.line(modeledandmeasured_melt2.loc[modeledandmeasured_melt2['position'] == 'Ap_5'], x = 'Datetime', y = 'value', color = 'datatype', facet_row = 'position', facet_col_wrap = 1, width = figwidth, height = figheight)
fig5.update_layout(
    plot_bgcolor='white',
    xaxis_title = "",
    yaxis_title = "15-min average irradiance (W/m<sup>2</sup>)",
    legend=dict(
        yanchor="top",
        y=1.2,
        xanchor="center",
        x=0.5
    ),
    legend_title=dict(
        text = "Data type"
    ),
    legend_orientation="h"
)
fig5.update_xaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey',
    title = " "
)
fig5.update_yaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey'
)
# for axis in fig1.layout:
#    if axis.startswith("xaxis"):
#       fig1.layout[axis].title = ""
fig5.show()

In [ ]:
type(fig1.layout)

Next steps:<br>
Divide the data by AM and PM and <br>
Implement 15-minute averaging to the data: dataframe.resample(freq='15T', label='right', closed='right').mean() <br>
Keep the data in wide form to avoid repeating indices <br>
Look for the script MBD_RMSE_functions.py and perform error analysis to annotatet the scatterplot (also ) <br>
Plot Irradiance sensor 3 vs PAR sensor (for within row and in control (GHI from SRRL)) and use the time of the day as a gradient and point shape as the data type <br>
Superimpose some of the figures with a side-view of an agrivoltaics row